Задание 3.

3.1. Посмотреть метрики вида (в библиотеке sklearn) Precision, Accuracy, Recall и F1 score. 

3.2. Понять где они наиболее приемлемы для использования. 

3.3. Сравнить результаты модели в различных метриках. Модель наилучшая после двух пунктов.

Ссылки:

1) Теория по метрикам кода (видеолекция на 8,5 мин.): https://courses.openedu.ru/courses/course-v1:mipt+DATA_AN+spring_2020/courseware/309ba8b3850e4278a9522d2c4e0e5135/4e2d182dbda54c3595a1632862257f4b/;

Задание 1.

1.1. Разобраться в функциях sklearn GridSearch, RandomSearch для дерева решений и логистической регрессии.

1.2. Сравнить результаты для обоих моделей. 

Возникали проблемы с установкой graphviz.

Возможное решение: https://overcoder.net/q/463149/python-%D0%BF%D1%80%D0%BE%D0%B1%D0%BB%D0%B5%D0%BC%D1%8B-%D1%81-%D1%83%D1%81%D1%82%D0%B0%D0%BD%D0%BE%D0%B2%D0%BA%D0%BE%D0%B9-pygraphviz-%D0%B8-graphviz

Проблема была в том, что при импорте с командной строки библиотека загружалась в папку с Питоном, не принадлежащим Анаконде

In [151]:
from IPython.display import SVG
from graphviz import Source
from IPython.display import display
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, RandomizedSearchCV

In [142]:
#Запуск случайных чисел
seed = 42
np.random.seed(seed)

# Чтение файла с обучающейся выборкой
data = pd.read_csv('../../datasets/in/titanic/train.csv')

# Присвоение переменным значений с количеством спасшихся и с остальными данными
y = data['Survived']
x = data.drop(['Survived'], axis=1)

# Функция по переводу строкового признака Пол в числовой
def binarize(x):
    if x == 'female':
        return 1
    else:
        return 0

# Перевод строкового признака Пол в числовой
x['Sex'] = x['Sex'].apply(lambda d: binarize(d))

# Добавление столбца с признаком удаленого события
x['Age_unk'] = x['Age'].apply(lambda d: np.isnan(d))

# Замена нулевых значений медианой
x['Age'].fillna(x['Age'].median(), inplace=True)


In [143]:
# Присвоение значений признаков для обучающей и тестовой выборке переменным
X_drop = x.copy()

In [144]:
# Подготовка данных с удалением нечисловых полей
select_column = ['PassengerId', 'Name', 'Ticket', 'Cabin', 'Embarked']
X_drop.drop(columns=select_column, inplace=True)

X_drop.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Age_unk
0,3,0,22.0,1,0,7.2500,False
1,1,1,38.0,1,0,71.2833,False
2,3,1,26.0,0,0,7.9250,False
3,1,1,35.0,1,0,53.1000,False
4,3,0,35.0,0,0,8.0500,False


In [152]:
# Подготовка данных для получения метрик
X_train, X_val, y_train, y_val = train_test_split(X_drop, y, test_size=.2, 
                                                  random_state=42)
model = LogisticRegression(random_state = 42)
parameters = {'C': np.logspace(-3,3,7),
              'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']}

# Получение метрик с использованием GridSearchCV
clf = GridSearchCV(model, parameters, cv=5, n_jobs=-1)
clf.fit(X_train, y_train)
y_preds = clf.best_estimator_.predict(X_val)
dummy_GS = {'Accuracy': accuracy_score(y_val, y_preds),
         'Precision': precision_score(y_val, y_preds),
         'Recall': recall_score(y_val, y_preds),
         'F1': f1_score(y_val, y_preds)}

# Получение метрик с использованием RandomSearch
clf = RandomizedSearchCV(model, parameters, cv=5, n_jobs=-1)
clf.fit(X_train, y_train)
y_preds = clf.best_estimator_.predict(X_val)
dummy_RS = {'Accuracy': accuracy_score(y_val, y_preds),
         'Precision': precision_score(y_val, y_preds),
         'Recall': recall_score(y_val, y_preds),
         'F1': f1_score(y_val, y_preds)}

In [153]:
dummy_GS

{'Accuracy': 0.7932960893854749,
 'Precision': 0.7761194029850746,
 'Recall': 0.7027027027027027,
 'F1': 0.7375886524822697}

In [154]:
dummy_RS

{'Accuracy': 0.7988826815642458,
 'Precision': 0.7794117647058824,
 'Recall': 0.7162162162162162,
 'F1': 0.7464788732394366}

Задание 2.

2.1. Взять исходный датасет Titanic и попробовать не выбрасывать строковые переменные, а закадировать их. 
Вам могут помочь LabelEncoder, OneHotEncoder и др, которые можно увидеть в соответствующих библиотеках sklearn. 
Сравнить полученные результаты со случаем, когда строковые переменные не использовались. 
И там и там используйте модель с гиперпараметрами, найденными в первом пункте.

Полезные ссылки:
    
1) Отличия LabelEncoder от OneHotEncoder: https://habr.com/ru/post/456294/

In [68]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [128]:
#Запуск случайных чисел
seed = 42
np.random.seed(seed)

# Чтение файла с обучающейся выборкой
data = pd.read_csv('../../datasets/in/titanic/train.csv')

# Присвоение переменным значений с количеством спасшихся и с остальными данными
y = data['Survived']
x = data.drop(['Survived'], axis=1)

# Функция по переводу строкового признака Пол в числовой
def binarize(x):
    if x == 'female':
        return 1
    else:
        return 0

# Перевод строкового признака Пол в числовой
x['Sex'] = x['Sex'].apply(lambda d: binarize(d))

# Добавление столбца с признаком удаленого события
x['Age_unk'] = x['Age'].apply(lambda d: np.isnan(d))

# Замена нулевых значений медианой
x['Age'].fillna(x['Age'].median(), inplace=True)



In [129]:
# Присвоение значений признаков для обучающей и тестовой выборке переменным
X_drop = x.copy()

X_encoder = x.copy()

In [130]:
# Подготовка данных с удалением нечисловых полей
# Удаление лишних (не числовых столбцов)
select_column = ['PassengerId', 'Name', 'Ticket', 'Cabin', 'Embarked']
X_drop.drop(columns=select_column, inplace=True)

X_drop.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Age_unk
0,3,0,22.0,1,0,7.2500,False
1,1,1,38.0,1,0,71.2833,False
2,3,1,26.0,0,0,7.9250,False
3,1,1,35.0,1,0,53.1000,False
4,3,0,35.0,0,0,8.0500,False


In [131]:
# Подготовка данных с кодированием полей с номером кабины и классом и удалением остальных нечисловых данных
select_column = ['PassengerId', 'Name', 'Ticket']
X_encoder.drop(columns=select_column, inplace=True)

encode = LabelEncoder()

X_encoder['Cabin'].fillna(101, inplace = True)
# X_encoder['Cabin'].fillna('unk', inplace = True)
# X_encoder['Cabin_unk'] = pd.Series(encode.fit_transform(X_encoder['Cabin'] == 'unk'))
X_encoder['Embarked'].fillna('S', inplace = True)
X_encoder['Embarked'] = pd.Series(encode.fit_transform(X_encoder['Embarked']))

X_encoder.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Age_unk
0,3,0,22.0,1,0,7.2500,101,2,False
1,1,1,38.0,1,0,71.2833,C85,0,False
2,3,1,26.0,0,0,7.9250,101,2,False
3,1,1,35.0,1,0,53.1000,C123,2,False
4,3,0,35.0,0,0,8.0500,101,2,False


In [132]:
# Деление данных на обучающие и валидационные

# Метод позволяющей поделить данные на 20 и 80%
X_drop_train, X_drop_test, y_train, y_test = train_test_split(X_drop,
                                                   y, test_size=0.2,
                                                   random_state=seed)

# Метод позволяющей поделить данные на 20 и 80%
X_encoder_train, X_encoder_test, y_train, y_test = train_test_split(X_encoder,
                                                   y, test_size=0.2,
                                                   random_state=seed)

In [134]:
# Получение метрик модели логистической регрессии с делением данных через kFold

kf = KFold(n_splits=5, shuffle=True)
result_X_drop = pd.DataFrame()

for num, idxs in enumerate(kf.split(X_drop_train)):
    print(f'Fold № {num}')
    train_index = idxs[0]
    val_index = idxs[1]
    X_train, X_val = X_drop_train.iloc[train_index], X_drop_train.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]
    model = LogisticRegression()
    model.fit(X_train, y_train)
    y_preds = model.predict(X_val)
    dummy = {'Accuracy': accuracy_score(y_val, y_preds),
             'Precision': precision_score(y_val, y_preds),
             'Recall': recall_score(y_val, y_preds),
             'F1': f1_score(y_val, y_preds)}
    result_X_drop = result_X_drop.append(dummy, ignore_index=True)

# --- Возникает ошибка - не может конвертировать одно из значений. 

# result_X_encoder = pd.DataFrame()
# for num, idxs in enumerate(kf.split(X_encoder_train)):
#     print(f'Fold № {num}')
#     train_index = idxs[0]
#     val_index = idxs[1]
#     X_train, X_val = X_encoder_train.iloc[train_index], X_encoder_train.iloc[val_index]
#     y_train, y_val = y.iloc[train_index], y.iloc[val_index]
#     model = LogisticRegression()
#     model.fit(X_train, y_train)
#     dummy = {'Accuracy': accuracy_score(y_val, y_preds),
#              'Precision': precision_score(y_val, y_preds),
#              'Recall': recall_score(y_val, y_preds),
#              'F1': f1_score(y_val, y_preds)}
#     result_X_encoder = result_X_encoder.append(dummy, ignore_index=True) 

Fold № 0
Fold № 1
Fold № 2
Fold № 3
Fold № 4


In [105]:
result_X_drop

,Accuracy,F1,Precision,Recall
0,0.615385,0.000000,0.000000,0.000000
1,0.580420,0.000000,0.000000,0.000000
2,0.619718,0.068966,1.000000,0.035714
3,0.584507,0.032787,1.000000,0.016667
4,0.661972,0.076923,0.666667,0.040816


In [95]:
result_X_encoder

""


In [135]:
# В попытке выяснить, в чем причина - удаляю закодированные поля
select_column = ['Embarked', 'Age_unk', 'Cabin']
X_encoder.drop(columns=select_column, inplace=True)

X_encoder.head()

,Pclass,Sex,Age,SibSp,Parch,Fare
0,3,0,22.0,1,0,7.2500
1,1,1,38.0,1,0,71.2833
2,3,1,26.0,0,0,7.9250
3,1,1,35.0,1,0,53.1000
4,3,0,35.0,0,0,8.0500


In [136]:
# Ошибка остается

kf_2 = KFold(n_splits=5, shuffle=True)
result_X_encoder = pd.DataFrame()
for num, idxs in enumerate(kf_2.split(X_encoder_train)):
    print(f'Fold № {num}')
    train_index = idxs[0]
    val_index = idxs[1]
    X_train, X_val = X_encoder_train.iloc[train_index], X_encoder_train.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]
    model = LogisticRegression()
    model.fit(X_train, y_train)
    dummy = {'Accuracy': accuracy_score(y_val, y_preds),
             'Precision': precision_score(y_val, y_preds),
             'Recall': recall_score(y_val, y_preds),
             'F1': f1_score(y_val, y_preds)}
    result_X_encoder = result_X_encoder.append(dummy, ignore_index=True) 

Fold № 0


ValueError: could not convert string to float: 'D26'